# Advanced Machine Learning (MScA, 32017)

# Project Paraphrase Detection

### Yuri Balasanov, Leonid Nazarov, Andrey Kobyshev, &copy; iLykei 2017

This project is based on [Quora Question Pairs](https://www.kaggle.com/c/quora-question-pairs) competition held on the Kaggle platform in spring 2017. It appeared to be one of the most popular competitions in the Kaggle history. <br>
The task was to detect duplicates among pairs of questions. <br>
The problem is also known as **paraphrase detection**.



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import isnan, when, count, length, lit, udf, col, struct
import numpy as np

In [2]:
spark = SparkSession.builder.getOrCreate()
spark

## Data structure

Look at the train and test data

In [3]:
trainFileName = "./data/quora_train_1000.csv"
testFileName = "./data/quora_test_1000.csv"

These files contain 1000 rows with data. <br>
Both these files are short versions of the project files. They can be analyzed in local spark environments. <br>

Create chema, read train data file, remove records with missing observations and cache it.

In [4]:
sch = StructType([StructField('id',IntegerType()), \
                  StructField('qid1',IntegerType()),\
                  StructField('qid2',IntegerType()), \
                  StructField('question1',StringType()),\
                  StructField('question2',StringType()), \
                  StructField('is_duplicate',IntegerType())])
train = spark.read.csv(trainFileName, header=True, escape='"', 
                       quote='"',schema=sch, multiLine = True)
train = train.dropna()

train.cache()
print('Number of rows = %s' % train.count())
train.show(6)

Number of rows = 1000
+---+----+----+--------------------+--------------------+------------+
| id|qid1|qid2|           question1|           question2|is_duplicate|
+---+----+----+--------------------+--------------------+------------+
|  0|   1|   2|What is the step ...|What is the step ...|           0|
|  1|   3|   4|What is the story...|What would happen...|           0|
|  2|   5|   6|How can I increas...|How can Internet ...|           0|
|  3|   7|   8|Why am I mentally...|Find the remainde...|           0|
|  4|   9|  10|Which one dissolv...|Which fish would ...|           0|
|  5|  11|  12|Astrology: I am a...|I'm a triple Capr...|           1|
+---+----+----+--------------------+--------------------+------------+
only showing top 6 rows



### Data fields  

*id* - the id of a training set question pair  
*qid1, qid2* - unique ids of each question (only available in train.csv)  
*question1, question2* - the full text of each question  
*is_duplicate* - the target variable, set to 1 if question1 and question2 have essentially the same meaning, and 0 otherwise.

See examples of non-duplicate and duplicate questions below.

Pair 5: non-duplicate

In [5]:
print(train.select('question1').collect()[4])
print(train.select('question2').collect()[4])

Row(question1='Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?')
Row(question2='Which fish would survive in salt water?')


Pair 6: duplicate

In [6]:
print(train.select('question1').collect()[5])
print(train.select('question2').collect()[5])

Row(question1='Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?')
Row(question2="I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?")


Read the test file, remove records with missing observations, cache it.

In [7]:
test = spark.read.csv(testFileName, header=True, escape='"', \
                            encoding='utf8', multiLine = True)
test = test.dropna()
test.cache()
print('Number of rows = %s' % test.count())
test.show(6)

Number of rows = 1000
+-------+--------------------+--------------------+
|test_id|           question2|           question1|
+-------+--------------------+--------------------+
|      0|How do I show tha...|Emoticons: What g...|
|      1|What is the scope...|Does ECE have a s...|
|      2|What was the orig...|Why do prosecuted...|
|      3|How  can someone ...|How do I grow tal...|
|      4|Can weapons to pa...|What is the bigge...|
|      5|What is the if I ...|What happens when...|
+-------+--------------------+--------------------+
only showing top 6 rows



Note that column *is_duplicate* is not present in test file.

For our purposes we need to join both dataframes. <br>
Drop columns that we don't need from `'train'`

In [8]:
train = train.drop('qid1', 'qid2')


Add to `test` a new column `id` which has integer values of `'test_id'` increased by maximum value of column `'id'` from `train`.
Then remove column `'test_id'` from test.

+--------------------+--------------------+----+ <br>
|           question2|           question1|  id| <br>
+--------------------+--------------------+----+ <br>
|How do I show tha...|Emoticons: What g...|1000| <br>
|What is the scope...|Does ECE have a s...|1001| <br>
|What was the orig...|Why do prosecuted...|1002| <br>
|How  can someone ...|How do I grow tal...|1003| <br>
|Can weapons to pa...|What is the bigge...|1004| <br>
+--------------------+--------------------+----+ <br>

**Hint.** Use 

`.groupBy().max()`

and 

`withColumn()`

as the following example shows.

In [9]:
# Create a toy dataframe
from pyspark.sql import Row
df = sqlContext.createDataFrame([ \
     Row(_1=0.0, _2=1.0), \
     Row(_1=1.0, _2=2.0), \
     Row(_1=2.0,_2=3.0), \
     Row(_1=3.0,_2=4.0)],["X","Y"])
df.show()

+---+---+
|  X|  Y|
+---+---+
|0.0|1.0|
|1.0|2.0|
|2.0|3.0|
|3.0|4.0|
+---+---+



In [18]:
# Find maximum of column Y
df.groupBy().max('Y').show()
maxY=df.groupBy().max('Y').collect()[0][0]
print('maxY = %s' % maxY)

+------+
|max(Y)|
+------+
|   4.0|
+------+

maxY = 4.0


In [16]:
df.groupBy().max('Y').collect()[0][0]

4.0

In [19]:
# Create new column
df.withColumn("new",(df.X+maxY).cast("integer")).show()

+---+---+---+
|  X|  Y|new|
+---+---+---+
|0.0|1.0|  4|
|1.0|2.0|  5|
|2.0|3.0|  6|
|3.0|4.0|  7|
+---+---+---+



Insert code in the following cell 

In [28]:
maxTrainID=train.groupBy().max('id').collect()[0][0]
test=test.withColumn("id",(test.test_id+maxTrainID+1).cast("integer")).drop('test_id')
test.show(5)

+--------------------+--------------------+----+
|           question2|           question1|  id|
+--------------------+--------------------+----+
|How do I show tha...|Emoticons: What g...|1000|
|What is the scope...|Does ECE have a s...|1001|
|What was the orig...|Why do prosecuted...|1002|
|How  can someone ...|How do I grow tal...|1003|
|Can weapons to pa...|What is the bigge...|1004|
+--------------------+--------------------+----+
only showing top 5 rows



In [122]:
maxTrainID = train.groupBy().max('id').collect()[0][0]
test = test.withColumn("id",(test.test_id+maxTrainID+1).cast("integer")).drop('test_id')
test.show(5)

+--------------------+--------------------+----+
|           question2|           question1|  id|
+--------------------+--------------------+----+
|How do I show tha...|Emoticons: What g...|1000|
|What is the scope...|Does ECE have a s...|1001|
|What was the orig...|Why do prosecuted...|1002|
|How  can someone ...|How do I grow tal...|1003|
|Can weapons to pa...|What is the bigge...|1004|
+--------------------+--------------------+----+
only showing top 5 rows



Add column `'is_duplicate'` containing vaues `-1`

In [29]:
test = test.withColumn('is_duplicate', lit(-1))
test.show(6)

+--------------------+--------------------+----+------------+
|           question2|           question1|  id|is_duplicate|
+--------------------+--------------------+----+------------+
|How do I show tha...|Emoticons: What g...|1000|          -1|
|What is the scope...|Does ECE have a s...|1001|          -1|
|What was the orig...|Why do prosecuted...|1002|          -1|
|How  can someone ...|How do I grow tal...|1003|          -1|
|Can weapons to pa...|What is the bigge...|1004|          -1|
|What is the if I ...|What happens when...|1005|          -1|
+--------------------+--------------------+----+------------+
only showing top 6 rows



Now join both dataframes.

In [30]:
data = train.union(test.select(train.columns))
print('Number of rows = %s' % data.count())
data.filter(data.id > 996).show(6)

Number of rows = 2000
+----+--------------------+--------------------+------------+
|  id|           question1|           question2|is_duplicate|
+----+--------------------+--------------------+------------+
| 997|I and my girlfrie...|Why most of the c...|           0|
| 998|Could we use cher...|Can we map the su...|           1|
| 999|What is a good so...|Diving the Blue H...|           0|
|1000|Emoticons: What g...|How do I show tha...|          -1|
|1001|Does ECE have a s...|What is the scope...|          -1|
|1002|Why do prosecuted...|What was the orig...|          -1|
+----+--------------------+--------------------+------------+
only showing top 6 rows



In [33]:
train.show(6)

+---+--------------------+--------------------+------------+
| id|           question1|           question2|is_duplicate|
+---+--------------------+--------------------+------------+
|  0|What is the step ...|What is the step ...|           0|
|  1|What is the story...|What would happen...|           0|
|  2|How can I increas...|How can Internet ...|           0|
|  3|Why am I mentally...|Find the remainde...|           0|
|  4|Which one dissolv...|Which fish would ...|           0|
|  5|Astrology: I am a...|I'm a triple Capr...|           1|
+---+--------------------+--------------------+------------+
only showing top 6 rows




## Python and Spark natural language processing tools

From the very beginnig Spark platform was created largely for processing large number of text documents. <br>
So, there is significant number of advanced tools for NLP.

Here is the list of the major Python Natural Language Processing (NLP) packages:

- [Natural Language Toolkit](http://www.nltk.org/) (NLTK)
- [TextBlob](https://textblob.readthedocs.io/en/dev/) is built on top of NLTK, and it's more easily-accessible.
- [Stanford's CoreNLP](https://stanfordnlp.github.io/CoreNLP/) is a Java library with Python wrappers. It presents in many existing production systems due to its speed.
- [SpaCy](https://spacy.io/) is a new NLP library that's designed to be fast, streamlined, and production-ready. It's not as widely adopted, but if you're building a new application, you should give it a try.
- [Gensim](https://radimrehurek.com/gensim/) is most commonly used for topic modeling and similarity detection. It's not a general-purpose NLP library, but for the tasks it does handle, it does them well.

Some of these tools are also available as part of Spark ML. <br>

Below there is a description of some important methods of creating features for natural text analysis.

## Tokenization

Tokenization is the procedure of chopping document into pieces, called **tokens**. <br>
In the process of tokenization some characters, such as punctuation, are usually removed. <br>
Tokens may be words or sentences. <br>

Below is an example from NLTK.

Download a popular collection of packages from *nltk* which includes some useful methods.

In [37]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\JohntheGreat\AppData\Roaming\nltk_data..
[nltk_data]    |     .
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\JohntheGreat\AppData\Roaming\nltk_data..
[nltk_data]    |     .
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\JohntheGreat\AppData\Roaming\nltk_data..
[nltk_data]    |     .
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\JohntheGreat\AppData\Roaming\nltk_data..
[nltk_data]    |     .
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\JohntheGreat\AppData\Roaming\nltk_data..
[nltk_

True

Methods *nltk.sent_tokenize* and *word_tokenize* belong to Penn Treebank Tokenizer module which is one module in the popular collection downloaded above.

Syntax:

`sent_tokenize(text, language='english')` <br>
`word_tokenize(text, language='english', preserve_line=False)` <br>

*sent_tokenize* returns sentense-tokenized copy of *text*. <br>
*word_tokenize* returns words-tokenized copy of *text*. Text may be previously sentense_tokenized or not. <br> Parameter *preserve_line=True* keeps words-tokenized sentenses separate in the list.

In [39]:
sents = nltk.sent_tokenize(data.filter(data.id == 3) \
                           .select('question1').collect()[0][0])
print('Tokens sentences: %s' % sents)
words = nltk.word_tokenize(sents[1])
print('Tokens words: %s' % words)
words1 = nltk.word_tokenize(data.filter(train.id == 3) \
                           .select('question1').collect()[0][0], \
                            preserve_line=False)
print('Tokens words1: %s' % words1)

Tokens sentences: ['Why am I mentally very lonely?', 'How can I solve it?']
Tokens words: ['How', 'can', 'I', 'solve', 'it', '?']
Tokens words1: ['Why', 'am', 'I', 'mentally', 'very', 'lonely', '?', 'How', 'can', 'I', 'solve', 'it', '?']


## Removing stopwords and punctuation

To keep only important words use the list of stopwords from NLTK.

In [40]:
stop_words = nltk.corpus.stopwords.words('english')
print(stop_words[:15])
stop_words = set(stop_words)
words = [w for w in words if not w.lower() in stop_words]
print(words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him']
['solve', '?']


Note that stopwords are converted to a set to make comparison faster. <br>

It may also be useful to remove punctuation. <br>
Function *s.isalpha()* tests if *s* is non-empty and all characters in *s* are alphabetic.

In [42]:
words = [w for w in words if w.isalpha()]
print(words)

['solve']


## POS-Tagging

Part-of-speech tagging (or POS-tagging) is the process of classifying words into their parts of speech and labeling them accordingly. 

Parts of speech are also known as **word classes** or **lexical categories**. 

Collection of tags used for particular task is called **tagset**.

In [129]:
words = nltk.word_tokenize(train.filter(train.id == 2). \
                           select('question1').collect()[0][0])
nltk.pos_tag(words)

[('How', 'WRB'),
 ('can', 'MD'),
 ('I', 'PRP'),
 ('increase', 'VB'),
 ('the', 'DT'),
 ('speed', 'NN'),
 ('of', 'IN'),
 ('my', 'PRP$'),
 ('internet', 'JJ'),
 ('connection', 'NN'),
 ('while', 'IN'),
 ('using', 'VBG'),
 ('a', 'DT'),
 ('VPN', 'NNP'),
 ('?', '.')]

The default tagger of nltk.pos_tag() uses the [Penn Treebank Tag Set](http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html). NLTK provides documentation for each tag, which can be queried using the tag or the beginning of it, e.g. 

In [43]:
nltk.download("tagsets")
nltk.help.upenn_tagset('N')

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\JohntheGreat\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
NNPS: noun, proper, plural
    Americans Americas Amharas Amityvilles Amusements Anarcho-Syndicalists
    Andalusians Andes Andruses Angels Animals Anthony Antilles Antiques
    Apache Apaches Apocrypha ...
NNS: noun, common, plural
    undergraduates scotches bric-a-brac products bodyguards facets coasts
    divestitures storehouses designs clubs fragrances averages
    subjectivists apprehensions muses factory-jobs ...


## Lemmatization and Stemming

The purpose of **lemmatization** and **stemming** is to standardize words to their root (stem) or their common morphological meaning or "minimal unit of meaning" (lemma).

[The Stanford NLP Group](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html) desribes these procedures as follows.  
The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. 

For instance:

am, are, is $\Rightarrow$ be  
car, cars, car's, cars' $\Rightarrow$ car  

Stemming usually refers to a crude heuristic process that cuts the ends and/or beginnings of words to extract their common roots.

Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma.  

The NLTK lemmatization method is based on [WordNet](http://wordnet.princeton.edu/)'s lemmatizer.  

Syntax of lemmatizer is:

`lemmatize(word,pos='n')`,

where parameter *pos* means part-of-speech and is defauled to noun, but can also be provided as other pos value.

In [44]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
words = ['dogs','having','lower','traditional']

print(list(map(wordnet_lemmatizer.lemmatize,words)))

['dog', 'having', 'lower', 'traditional']


Lemmatizer did not find correct lemmas for 'having' and 'lower'. We can fix this issue setting the second parameter of the fuction lemmatize - 'pos' . 

The appropriate 'pos' values for noun, verb, adjective and adverb are 

In [45]:
from nltk.corpus import wordnet as wn
print(wn.NOUN,wn.VERB,wn.ADJ,wn.ADV)

n v a r


while the default value is 'n'. 

With correct 'pos' we get correct lemmas

In [46]:
(wordnet_lemmatizer.lemmatize('having','v'),
wordnet_lemmatizer.lemmatize('lower','a'))

('have', 'low')

In [65]:
print(wordnet_lemmatizer.lemmatize('understand','n'))
print(wordnet_lemmatizer.lemmatize(wordnet_lemmatizer.lemmatize(('traditional'),'a'),'v'))
print(wordnet_lemmatizer.lemmatize('traditional','r'))

understand
traditional
traditional


It is recommended to POS-tagging before lemmatizing.  

NLTK provides several well known stemmers interfaces, such as [Porter stemmer](https://tartarus.org/martin/PorterStemmer/), Lancaster Stemmer, [Snowball Stemmer](http://snowballstem.org/) etc. 

The following example shows how to use a stemmer.

In [59]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
print([stemmer.stem(w) for w in words])

['dog', 'have', 'lower', 'tradit']


Compare the word 'traditional' transformation by stemmer and lemmatizer.

<font color=blue>

Create function `lemmas_nltk(s):` which: <br>
- Takes sentense `s` as argument 
- Splits it into words and transforms each word into lower case
- Lemmatizes each word as verb and as noun
- Removes stopwords, non-alphabetic words

**Hint.** <br>

Use `.join()`, `.lower()`, `.split()`, `.isalpha()`.

To check the function run the code:

`ss=train.select('question2').take(2)[1][0]` <br>
`print(ss)` <br>
`print(lemmas_nltk(ss))` <br>

To receive the result:

`What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?` <br>
`would happen indian government steal kohinoor diamond` <br>

Enter code in the following cell.

In [92]:
#Skipped code
#Function lemmas_nltk(s)

# split, lower, remove stopwords and non alpha, then lemmatize

def lemmas_nltk(s):
    return " ".join([wordnet_lemmatizer.lemmatize(wordnet_lemmatizer.lemmatize(w,'n'),'v') \
                     for w in s.lower().split() if w.isalpha() & (not w in stop_words)])

lemmas_nltk("What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?")

'would happen indian government steal kohinoor diamond'

<font color=blue>

Create user defined function from function `lemmas_nltk(s)` to apply it to new column <br>

Function syntax

`pyspark.sql.functions.udf(f=None, returnType=StringType)`

Function creates a Column expression representing a user defined function (UDF).

The user-defined functions must be deterministic. 
Parameters:	

`f – python function if used as a standalone function`
`returnType – a pyspark.sql.types.DataType object`

In [93]:
#Skipped code
lemmas_nltk_udf = udf(lemmas_nltk, StringType())

 Apply function `lemmas_nltk_udf` to a column of second question in train sample.

In [114]:
#ss=train.select('question2')
#print(ss.take(2)[1][0])
ss.show()
ss2=ss.withColumn('question3', lemmas_nltk_udf(ss.question2))
ss2.show()


+--------------------+
|           question2|
+--------------------+
|What is the step ...|
|What would happen...|
|How can Internet ...|
|Find the remainde...|
|Which fish would ...|
|I'm a triple Capr...|
|What keeps childe...|
|What should I do ...|
|When do you use "...|
|How do I hack Mot...|
|What are some of ...|
|How can I see all...|
|How can you make ...|
|What was your fir...|
|What are the laws...|
|How will a Trump ...|
|What does manipul...|
|How do guys feel ...|
|Why do people ask...|
|Which is the best...|
+--------------------+
only showing top 20 rows

+--------------------+--------------------+
|           question2|           question3|
+--------------------+--------------------+
|What is the step ...|step step guide i...|
|What would happen...|would happen indi...|
|How can Internet ...|internet speed in...|
|Find the remainde...|find remainder di...|
|Which fish would ...|fish would surviv...|
|I'm a triple Capr...|triple capricorn ...|
|What keeps childe...|keep

<font color=blue>

Create function `wordsCount(str):` which returns word count of string, plus 1

Check the function by running code:

`wordsCount('I love data')`

The answer should be 3

Create wordsCount_udf using `.udf()` of type integer

In [120]:
#Skipped code
# wordsCount() 
def wordsCount(str):
    return str.count(' ')+1
#wordsCount_udf
wordsCount_udf = udf(wordsCount, IntegerType())
wordsCount('I love data')

3

 <font color=blue>

Create function `ratio(x,y):` which calculates
$$\frac{|x-y|}{x+y}.$$ 
Can you make with respect to division by zero?

Create `ratio_udf` using `.udf()` of type double

In [121]:
def ratio(x,y): return abs(x-y)/(x+y+1e-15) ############## divide by zero occurs!!! ###########################
ratio_udf = udf(ratio, DoubleType())

## Semantic similarity

It is important to have a measure of distance between words based on the similarity of their meaning or semantic content as opposed to similarity of their syntactical representation (e.g. their string format or word count).  

WordNet created in Cognitive Science Laboratory pf Princeton University gives tools for measuring semantic similarity. 

Nouns, verbs, adjectives and adverbs in this database are grouped into cognitive synonym sets (**synsets**). 

The most frequently encoded relation among synsets is the **super-subordinate relation** (also called **hyperonymy**, **hyponymy** or **ISA relation**). 

WordNet links more general synsets like *furniture, piece_of_furniture* to increasingly specific ones like *bed* and *bunkbed*. 

See more detailed description on the [WordNet](https://wordnet.princeton.edu/) page or [here](https://arxiv.org/ftp/arxiv/papers/1310/1310.8059.pdf). 

In [141]:
print(wn.synsets('dog'))

[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01'), Synset('chase.v.01')]


A synset is identified with a 3-part name of the form: word.pos.nn.

Function *synset1.path_similarity(synset2)* returns a score denoting how similar two word senses are, based on the shortest path that connects the senses in the is-a (hypernym/hyponym) taxonomy. The score is in the range 0 to 1. Large score means similar words. 

In [142]:
dog = wn.synset('dog.n.01')
pet = wn.synset('pet.n.01')
table = wn.synset('table.n.01')
print('Similarity of dog to pet = %s' % dog.path_similarity(pet))
print('Similarity of dog to table = %s' % dog.path_similarity(table))
print('Similarity of dog to horse = %s' % \
      wn.synsets('dog')[0].path_similarity(wn.synsets('horse')[0]))

Similarity of dog to pet = 0.25
Similarity of dog to table = 0.07142857142857142
Similarity of dog to horse = 0.125


Path Similarity is not the only mesure implemented in WordNet. 

There are, for example, **Leacock-Chodorow Similarity**, **Wu-Palmer Similarity**, **Resnik Similarity**, **Jiang-Conrath Similarity**, **Lin Similarity** (see descriptions [here](https://arxiv.org/ftp/arxiv/papers/1310/1310.8059.pdf)).

## Word2Vec

Word2vec is a mapping of words from a vocabulary to vectors of real numbers. 

This mapping was [created](https://arxiv.org/pdf/1301.3781.pdf) using neural networks by a team of researchers led by Tomas Mikolov at Google.  

The purpose of Word2vec is to group the vectors of similar words together in vectorspace using mathematical similarities. 

Word2vec creates vectors that are distributed numerical representations of word features, such as the context of individual words.

Word2Vec model depends on the corpora using which the neural network was trained. 

Below we use Google’s pre-trained Word2Vec neural network model that can be downloaded [here](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit). Its size is 1.5GB. 

This model includes word vectors for a vocabulary of 3 million words and phrases based on training set of roughly 100 billion words from  Google News dataset. Each vector contains 300 features. 

Load the dataset. This requires package *gensim* which can be [installed](https://anaconda.org/anaconda/gensim) by running 

*conda install -c anaconda gensim*

in terminal.

In [143]:
import gensim
model = gensim.models.KeyedVectors \
.load_word2vec_format('/home/yuri/projects/Quora/GoogleNews-vectors-negative300.bin.gz',
                      binary=True)

The model is able to capture multiple different degrees of similarity between words. Semantic and syntactic patterns can be reproduced using vector arithmetic. Patterns such as “Queen is to Woman as King is to Man” can be generated through algebraic operations on the vector representations of these words such that the vector representation of “King” - ”Man” + ”Woman” produces a result which is closest to the vector representation of “Queen” in the model. Such relationships can be generated for a range of semantic relations (such as Country—Capital)

In [144]:
print(model.most_similar(positive=['capital','Iceland'])[0])
print(model.most_similar(positive=['woman', 'king'],
                         negative=['man'])[0])

('Reykjavik', 0.6050446629524231)
('queen', 0.7118192911148071)


Find most similar word to 'Oz' and 'Toto'. <br>
Find most similar word to 'Dorothy' and 'Toto'. <br>
Can it recognize that positive association with 'Garland' (Judy) and 'Toto' (the dog) and negative association with 'Hamilton' (Margaret, who played Wicked Witch of the West) should be Dorothy? 

In [145]:
print(model.most_similar(positive=['Oz', 'Toto'])[0])
print(model.most_similar(positive=['Dorothy', 'Toto'])[0])
print(model.most_similar(positive=['Garland', 'Toto'],
                         negative=['Hamilton'])[0])

('One_worshiper_Yehuda', 0.5837857127189636)
('Bernice', 0.6208423376083374)
('Pangako', 0.36821073293685913)


No, we expect too much from this model.

This model enables vector presentation of the question. 
If *words*  is a list of question's words after removing stopwords then the vector representation *question2vec* can be obtained as follows

In [146]:
print('Words: %s' %words)
M = []
for w in words:
    try: M.append(model[w])
    except: continue
M = np.array(M)
if len(M)==0: question2vec = np.zeros(300)
else: 
    question2vec = M.sum(axis=0)
    norm = np.sqrt((question2vec ** 2).sum())
    if norm>0: question2vec /= norm 

print(question2vec[:5])
len(question2vec)

Words: ['dogs', 'having', 'lower', 'traditional']
[-0.02835612 -0.01235805 -0.0893375   0.1272655  -0.03042328]


300

Try to run the same code with only first word in *words*, i.e. *words[0]*. The returned vector is different, but also has length of 300.

Any distance, like cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis etc. between question's vector representations may be used as measure of their similarity.

## TF-IDF

One more way of obtaining vector representations is using **Term frequency-inverse document frequency** ([TF-IDF](http://spark.apache.org/docs/2.2.0/ml-features.html#tf-idf)) methodolody implemented in `pyspark.ml.feature` package.
The scheme consists of the following steps.  
(1) Create the corpus of all questions from train and test datasets.  
(2) Initialize class `CountVectorizer`.  
(3) Fit a `CountVectorizerModel` to this corpus.  
(4) Apply `CountVectorizerModel.transform()` to *train.question1, train.question2, test.question1, test.question2*.

In [147]:
from pyspark.ml.feature import IDF, Tokenizer, CountVectorizer

Create a toy example

In [148]:
corpus = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("Logistic regression models are neat".split(" "), ),
    ("I wish Java could use case classes".split(" "), )
    ], ["text"])
corpus.show(truncate=False)

+------------------------------------------+
|text                                      |
+------------------------------------------+
|[Hi, I, heard, about, Spark]              |
|[Logistic, regression, models, are, neat] |
|[I, wish, Java, could, use, case, classes]|
+------------------------------------------+



 Initialize a `CountVectorizer`

In [149]:
cv = CountVectorizer(inputCol="text", outputCol="vectors", minDF=1.0)

Syntax of CountVectorizer:

`CountVectorizer(minTF=1.0, minDF=1.0, vocabSize=262144, binary=False, inputCol=None, outputCol=None)`

* `minTF` Filter to ignore rare words in a document. For each document, terms with frequency/count less than the given threshold are ignored. If this is an integer >= 1, then this specifies a count (of times the term must appear in the document); if this is a double in [0,1), then this specifies a fraction (out of the document's token count).
* `minDF` Specifies the minimum number of different documents a term must appear in to be included in the vocabulary
* `vocabSize` max size of the vocabulary. Default 1e18

Fit a CountVectorizerModel

In [150]:
cvModel = cv.fit(corpus)
cvModel.vocabulary

['I',
 'wish',
 'use',
 'Spark',
 'could',
 'about',
 'heard',
 'regression',
 'Hi',
 'are',
 'Logistic',
 'case',
 'classes',
 'Java',
 'neat',
 'models']

Apply the model. <br>

Column *vectors* contains number of words in the created vocabulary, list of word IDs in the vocabulary and real-valued vectors of frequences.

In [151]:
corpus = cvModel.transform(corpus)
corpus.show(truncate=False)

+------------------------------------------+-----------------------------------------------------+
|text                                      |vectors                                              |
+------------------------------------------+-----------------------------------------------------+
|[Hi, I, heard, about, Spark]              |(16,[0,3,5,6,8],[1.0,1.0,1.0,1.0,1.0])               |
|[Logistic, regression, models, are, neat] |(16,[7,9,10,14,15],[1.0,1.0,1.0,1.0,1.0])            |
|[I, wish, Java, could, use, case, classes]|(16,[0,1,2,4,11,12,13],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
+------------------------------------------+-----------------------------------------------------+



Compute the Inverse Document Frequency

In [152]:
idf = IDF(inputCol="vectors", outputCol="idf")
idfModel = idf.fit(corpus)
corpus = idfModel.transform(corpus)

In [153]:
corpus.select('idf').take(2)

[Row(idf=SparseVector(16, {0: 0.2877, 3: 0.6931, 5: 0.6931, 6: 0.6931, 8: 0.6931})),
 Row(idf=SparseVector(16, {7: 0.6931, 9: 0.6931, 10: 0.6931, 14: 0.6931, 15: 0.6931}))]

Note that word 'I' appears twice in the corpus, so its weight is only half of weights of other words which appear once.

Now create features calculating different characteristics of each representation and distances between questions' representations.  
For examle `squared_distance`

In [154]:
idf1 = corpus.select('idf').collect()[0][0]
idf2 = corpus.select('idf').collect()[1][0]
idf1.squared_distance(idf2)

4.4068381000739647

## n-grams

n-gram is a contiguous sequence of n items (letters, words etc.) from a given sequence of text. <br>
An n-gram of size 1 is referred to as a "unigram"; size 2 is a "bigram". <br>
The number of common n-grams in the pairs of texts is a natural measure of their syntactic similarity. <br>
Counting common n-grams is easy in NLTK. <br>
Count bigrams for the first pair.

In [155]:
i = 0
row_i = data.filter(data.id == i).collect()[0]
quest1, quest2 = row_i.question1, row_i.question2

In [156]:
print(set(nltk.ngrams(nltk.word_tokenize(quest1), 2)))
commonBigrams = (set(nltk.ngrams(nltk.word_tokenize(quest1), 2)) &
     set(nltk.ngrams(nltk.word_tokenize(quest2), 2)))
print(commonBigrams)
print(len(commonBigrams))

{('step', 'by'), ('What', 'is'), ('step', 'guide'), ('the', 'step'), ('share', 'market'), ('india', '?'), ('guide', 'to'), ('to', 'invest'), ('market', 'in'), ('in', 'share'), ('is', 'the'), ('invest', 'in'), ('in', 'india'), ('by', 'step')}
{('step', 'by'), ('What', 'is'), ('step', 'guide'), ('the', 'step'), ('share', 'market'), ('guide', 'to'), ('to', 'invest'), ('in', 'share'), ('is', 'the'), ('invest', 'in'), ('by', 'step')}
11


The number of letter bigrams for the same pair is

In [157]:
print(len(set(nltk.ngrams(quest1, 2)) &
     set(nltk.ngrams(quest2, 2))))

39


<font color=blue>

Create function `commonNgrams(s1, s2, n):` which:

1. Takes 2 documents or lemmatized documents `s1` and `s2` and length of n-gram `n`
2. Splits them and transforms them into lower case
3. Removes unnecessary special symbols by compiling '([^\s\w]|_)+'
4. Calculates and returns number of common n-grams

**Hint.** 

1. Use functions `.lower()`, `.split()`
2. Import `re` and use `re.compile()` and `re.sub()` to remove special symbols

Python function

`re.compile(pattern, flags=0)`

compiles a regular expression pattern into a regular expression object, which can be used for matching using its match() and search() methods, described below.

The expression’s behaviour can be modified by specifying a flags value. Values can be any of the following variables, combined using bitwise OR (the | operator).

The sequence

`prog = re.compile(pattern)`
`result = prog.match(string)`

is equivalent to

`result = re.match(pattern, string)`

but using re.compile() and saving the resulting regular expression object for reuse is more efficient when the expression will be used several times in a single program.

Python function `re.sub()` removes or substitutes one pattern with another.

Example:

In [158]:
import re
regex = re.compile('\w')
print(regex.sub('', 'X + Y - Z^2 = 5'))

 +  - ^ = 


<font color=blue>

Create `commonNgrams_udf` by using `.udf()` with type integer

Check `commonNgrams(s1, s2, n)` by running code

`commonNgrams('I love data', 'I love MScA', 2)`

Result should be 1

In [159]:
#Skipped code
# commonNgrams(s1,s2,n), commonNgrams_udf
import re


<font color=blue>

Create function `unigram_ratio(ngrams, n1, n2):` which calculates unigram ratio by formula
$$\frac{ngrams}{1+\max(n_1,n_2)},$$
where `ngrams` is the number  of common grams and `$n_1$`,`$n_2$` are the word counts in 2 documents.

To check the function call it using the following example from Zeno's paradox: 

ss1='Tortoise runs slow' <br>
ss2='Achilles runs very fast' <br>
ngr=commonNgrams(ss1,ss2,1) <br>
print('ngr = %s' % ngr) <br>
print(ss1.count(' ')) <br>
print(ss2.count(' ')) <br>

<font color=blue>

Result should be 0.25 <br>

Enter code in the following cell

In [160]:
#Skipped code
#unigram_ratio()


## Fuzzy string matching

Sometimes texts are apparently duplicated but differ by a word or punctuation mark. <br>
Here is an example

In [161]:
i = 13
row_i = train.filter(train.id == i).collect()[0]
q1, q2, is_dupl = row_i.question1, row_i.question2, row_i.is_duplicate
print(q1,"\n",q2,"\n",is_dupl)

What was your first sexual experience like? 
 What was your first sexual experience? 
 1


Package [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) can handle well such cases. It uses [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance) to calculate the differences between sequences. Functions **ratio, partial_ratio, token_sort_ratio** and **token_set_ratio** return similarity score from 0 to 100. Let us see how do they score the pair above.

[Install](https://anaconda.org/conda-forge/fuzzywuzzy) the package by running in terminal:

*conda install -c conda-forge fuzzywuzzy*

In [162]:
import warnings
warnings.filterwarnings('ignore')

In [163]:
from fuzzywuzzy import fuzz
print("Simple Ratio = ",fuzz.ratio(q1,q2))
print("Partial Ratio = ",fuzz.partial_ratio(q1,q2))
print("Token Sort Ratio = ",fuzz.token_sort_ratio(q1,q2))
print("Token Set Ratio = ",fuzz.token_set_ratio(q1,q2))

Simple Ratio =  94
Partial Ratio =  97
Token Sort Ratio =  94
Token Set Ratio =  100


Another Python module providing classes and functions for comparing sequences is [difflib](https://docs.python.org/3/library/difflib.html). Function **ratio** return a measure of the sequences’ similarity as a float in the range [0, 1]. 

In [164]:
import difflib
seq = difflib.SequenceMatcher()
seq.set_seqs(q1.lower(), q2.lower())
print(seq.ratio())

0.9382716049382716


# Features creating example

Use `pyspark.sql.functions.length` function to create new columns with lengths of questions

In [165]:
from pyspark.sql.functions import length
for i in ['1', '2']:
    data = data.withColumn('len'+i, length(data['question'+i]))
data.show(5)

+---+--------------------+--------------------+------------+----+----+
| id|           question1|           question2|is_duplicate|len1|len2|
+---+--------------------+--------------------+------------+----+----+
|  0|What is the step ...|What is the step ...|           0|  66|  57|
|  1|What is the story...|What would happen...|           0|  51|  88|
|  2|How can I increas...|How can Internet ...|           0|  73|  59|
|  3|Why am I mentally...|Find the remainde...|           0|  50|  65|
|  4|Which one dissolv...|Which fish would ...|           0|  76|  39|
+---+--------------------+--------------------+------------+----+----+
only showing top 5 rows



<font color=blue>

Create new column `lenRatio` with user-defined *function ratio_udf*

Running code 

`data.select('question1','question2','len1','len2','lenRatio').show(5)`

should show

+--------------------+--------------------+----+----+-------------------+ <br>
|           question1|           question2|len1|len2|           lenRatio| <br>
+--------------------+--------------------+----+----+-------------------+ <br>
|What is the step ...|What is the step ...|  66|  57|0.07317073170731707| <br>
|What is the story...|What would happen...|  51|  88|0.26618705035971224| <br>
|How can I increas...|How can Internet ...|  73|  59|0.10606060606060606| <br>
|Why am I mentally...|Find the remainde...|  50|  65|0.13043478260869565| <br>
|Which one dissolv...|Which fish would ...|  76|  39| 0.3217391304347826| <br>
+--------------------+--------------------+----+----+-------------------+ <br>

In [166]:
#Skipped code
#LenRatio


## Comment on test set

As an anti-cheating measure, Kaggle has supplemented the test set with computer-generated question pairs. Those rows do not come from Quora, and are not counted in the scoring. All of the questions in the training set are genuine examples from Quora.  
The test set you get for this project is different from Quora *test.csv* but it also contains a lot of pairs not counted in the scoring.  The *[train.csv](https://www.kaggle.com/c/quora-question-pairs/download/train.csv.zip)* is the original Quora train set, it can be downloaded from [competition page](https://www.kaggle.com/c/quora-question-pairs/data).

## Project description

For the first part of this project create the following set of features:

In [167]:
featureNames = ['Lemmas1', 'Lemmas2',
                'rawWords1', 'rawWords2',
                'Len1', 'Len2',
                'rawLen1', 'rawLen2',
                'diff_lemmas_ratio', 'diff_rawWords_ratio',
                'diff_len_ratio', 'diff_rawLen_ratio',
                'raw_ngrams_1', 'raw_ngrams_2', 'raw_ngrams_3', 
                'ngrams_1', 'ngrams_2', 'ngrams_3', 
                'raw_unigram_ratio', 'unigram_ratio', 
                'tfidfDistance']

### Lemmatization

<font color=blue>

Create lemma string for each of the questions using function `lemmas_nltk_udf`

+---+----+----+--------------------+--------------------+--------------------+ <br>
| id|len1|len2|            lenRatio|              lemma1|              lemma2| <br>
+---+----+----+--------------------+--------------------+--------------------+ <br>
|  0|  66|  57| 0.07317073170731707|step step guide i...|step step guide i...| <br>
|  1|  51|  88| 0.26618705035971224|      story kohinoor|would happen indi...| <br>
|  2|  73|  59| 0.10606060606060606|increase speed in...|internet speed in...| <br>
|  3|  50|  65| 0.13043478260869565|      mentally solve|find remainder di...| <br>
|  4|  76|  39|  0.3217391304347826|one dissolve wate...|fish would surviv...| <br>
|  5|  86|  90|0.022727272727272728|capricorn sun cap...|triple capricorn ...| <br>
+---+----+----+--------------------+--------------------+--------------------+ <br>

In [168]:
#Skipped code
#Lemmas


### Questions length, lemmas length

<font color=blue>

Create columns of lemmas counts: 

- `Lemmas1`, `Lemmas2`: word counts in lemmas
- `rawWords1`, `rawWords2`:  word counts in original questions
- `Len1`, `Len2`: lengths of lemmas
- `rawLen1`, `rawLen2`: lengths of questions.

Use functions `wordsCount_udf`.

Selection by code

`data.select('Lemmas1','Lemmas2','rawWords1','rawWords2','Len1','Len2','rawLen1','rawLen2').show(6)`

Should look like:

+-------+-------+---------+---------+----+----+-------+-------+ <br>
|Lemmas1|Lemmas2|rawWords1|rawWords2|Len1|Len2|rawLen1|rawLen2| <br>
+-------+-------+---------+---------+----+----+-------+-------+ <br>
|      6|      5|       14|       12|  35|  28|     66|     57| <br>
|      2|      7|        8|       13|  14|  53|     51|     88| <br>
|      5|      4|       14|       10|  38|  28|     73|     59| <br>
|      2|      3|       11|        9|  14|  21|     50|     65| <br>
|      7|      4|       13|        7|  43|  23|     76|     39| <br>
|      6|      5|       16|       16|  30|  35|     86|     90| <br>
+-------+-------+---------+---------+----+----+-------+-------+ <br>

In [169]:
for i in ["1","2"]:
    data = data.withColumn('Lemmas'+i, wordsCount_udf(data['lemma'+i]))
    data = data.withColumn('rawWords'+i, wordsCount_udf(data['question'+i]))
    data = data.withColumn('Len'+i, length(data['lemma'+i]))
    data = data.withColumn('rawLen'+i, length(data['question'+i]))
data.select('Lemmas1','Lemmas2','rawWords1','rawWords2','Len1','Len2','rawLen1','rawLen2').show(6)

NameError: name 'wordsCount_udf' is not defined

### Lengths ratios

<font color=blue>

Create columns of lengths ratios using `ratio_udf`:

- `diff_lemmas_ratio`: ratio of `Lemmas1` and `Lemmas2`
- `diff_rawWords_ratio`: ratio of `rawWords1`and `rawWords2`
- `diff_len_ratio`: ratio of `Len1` and `Len2`
- `diff_rawLen_ratio`: ratio of `rawLen1` and `rawLen2`

Running

`data.select('diff_lemmas_ratio','diff_rawWords_ratio','diff_len_ratio','diff_rawLen_ratio').show(6)`

should show

+-------------------+-------------------+-------------------+--------------------+ <br>
|  diff_lemmas_ratio|diff_rawWords_ratio|     diff_len_ratio|   diff_rawLen_ratio| <br>
+-------------------+-------------------+-------------------+--------------------+ <br>
| 0.0909090909090909|0.07692307692307693| 0.1111111111111111| 0.07317073170731707| <br>
| 0.5555555555555555|0.23809523809523808|  0.582089552238806| 0.26618705035971224| <br>
|0.11111111111111109|0.16666666666666666|0.15151515151515152| 0.10606060606060606| <br>
|0.19999999999999996|                0.1|                0.2| 0.13043478260869565| <br>
| 0.2727272727272727|                0.3|0.30303030303030304|  0.3217391304347826| <br>
| 0.0909090909090909|                0.0|0.07692307692307693|0.022727272727272728| <br>
+-------------------+-------------------+-------------------+--------------------+ <br>

In [ ]:
#Skipped code
#Lengths ratios


### n-grams

<font color=blue>

Create features:

- `raw_ngrams_1`, `raw_ngrams_2`, `raw_ngrams_3`: n-grams of questions
- `ngrams_1`, `ngrams_2`, `ngrams_3`: n-grams of lemmas
- `raw_unigram_ratio`: unigram ratio for questions
- `unigram_ratio`: unigram ratio for lemmas

Use function `commonNgrams_udf`

Code

`data.select('ngrams_1','ngrams_2','ngrams_3', \` <br>
            `'raw_ngrams_1','raw_ngrams_2','raw_ngrams_3', \` <br>
            `'raw_unigram_ratio','unigram_ratio').show(6)` <br>
            
should show

+--------+--------+--------+------------+------------+------------+-------------------+-------------------+ <br>
|ngrams_1|ngrams_2|ngrams_3|raw_ngrams_1|raw_ngrams_2|raw_ngrams_3|  raw_unigram_ratio|      unigram_ratio| <br>
+--------+--------+--------+------------+------------+------------+-------------------+-------------------+ <br>
|       4|       4|       3|          11|          11|          10| 0.7333333333333333| 0.5714285714285714| <br>
|       1|       0|       0|           4|           2|           1| 0.2857142857142857|              0.125| <br>
|       3|       0|       0|           4|           1|           0|0.26666666666666666|                0.5| <br>
|       0|       0|       0|           0|           0|           0|                0.0|                0.0| <br>
|       0|       0|       0|           4|           0|           0| 0.2857142857142857|                0.0| <br>
|       3|       0|       0|           9|           4|           1| 0.5294117647058824|0.42857142857142855| <br>
+--------+--------+--------+------------+------------+------------+-------------------+-------------------+ <br>

In [ ]:
#Skipped code
#N-grams and n-gram ratios
